In [1]:
import os
import csv
import time
import datetime
from tqdm import tqdm
import numpy as np
from itertools import islice

In [2]:
def CSVWriter(outputfile, row):
    csvfile = open(outputfile, 'a+')
    writer = csv.writer(csvfile)
    writer.writerows(row)
    csvfile.close()
    
def getTimeDiff(timeStra, timeStrb):
    if timeStra==timeStrb:
        return 0
    ta = time.strptime(timeStra, "%Y-%m-%d %H:%M:%S")
    tb = time.strptime(timeStrb, "%Y-%m-%d %H:%M:%S")
    y,m,d,H,M,S = ta[0:6]
    dataTimea=datetime.datetime(y,m,d,H,M,S)
    y,m,d,H,M,S = tb[0:6]
    dataTimeb=datetime.datetime(y,m,d,H,M,S)
    secondsDiff=(dataTimeb-dataTimea).seconds
    return secondsDiff

def sort_acc_by_time(csv_reader):
    sortedata = sorted(csv_reader, key = lambda row: datetime.datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S"))
    return sortedata

def sort_qua_by_time(csv_reader):
    sortedata = sorted(csv_reader, key = lambda row: datetime.datetime.strptime(row[4], "%Y-%m-%d %H:%M:%S"))
    return sortedata
    
def pepa_accerlation(input_file, output_file, timestep, initial_time = '2018-10-12 23:00:00'):
    results = []
    with open(input_file) as fin:
        reader = csv.reader(fin)
        headers = reader.next()
        reader.next()
        reader.next()
        reader = sort_acc_by_time(reader)
        time1 = initial_time
        x = []
        y = []
        z = []
        for record in reader:
            record_dict = dict(zip(headers, record))
            if getTimeDiff(time1, record_dict["time"]) < timestep:
                x.append(float(record_dict["pepa_acc_x"]))
                y.append(float(record_dict["pepa_acc_y"]))
                z.append(float(record_dict["pepa_acc_z"]))
            else:
                sub_result = []
                sub_result.append(np.mean(x))
                sub_result.append(np.mean(y))
                sub_result.append(np.mean(z))
                sub_result.append(time1)
                results.append(sub_result)
                del x[:]
                del y[:]
                del z[:]
                x.append(float(record_dict["pepa_acc_x"]))
                y.append(float(record_dict["pepa_acc_y"]))
                z.append(float(record_dict["pepa_acc_z"]))
                time1 = record_dict["time"]
    CSVWriter(output_file, [['pepa_acc_x', 'pepa_acc_y', 'pepa_acc_z', 'time'], ['float', 'float', 'float', 'datetime'], ['', '', '', 'T']])
    CSVWriter(output_file, results)
    
def salt_accerlation(input_file, output_file, timestep, initial_time = '2018-10-12 23:00:00'):
    results = []
    with open(input_file) as fin:
        reader = csv.reader(fin)
        headers = reader.next()
        reader.next()
        reader.next()
        reader = sort_acc_by_time(reader)
        time1 = initial_time
        x = []
        y = []
        z = []
        for record in reader:
            record_dict = dict(zip(headers, record))
            if getTimeDiff(time1, record_dict["time"]) < timestep:
                x.append(float(record_dict["salt_acc_x"]))
                y.append(float(record_dict["salt_acc_y"]))
                z.append(float(record_dict["salt_acc_z"]))
            else:
                sub_result = []
                sub_result.append(np.mean(x))
                sub_result.append(np.mean(y))
                sub_result.append(np.mean(z))
                sub_result.append(time1)
                results.append(sub_result)
                del x[:]
                del y[:]
                del z[:]
                x.append(float(record_dict["salt_acc_x"]))
                y.append(float(record_dict["salt_acc_y"]))
                z.append(float(record_dict["salt_acc_z"]))
                time1 = record_dict["time"]
    CSVWriter(output_file, [['salt_acc_x', 'salt_acc_y', 'salt_acc_z', 'time'], ['float', 'float', 'float', 'datetime'], ['', '', '', 'T']])
    CSVWriter(output_file, results)
    
def pepa_quaternion(input_file, output_file, timestep, initial_time = '2018-10-12 23:00:00'):
    results = []
    with open(input_file) as fin:
        reader = csv.reader(fin)
        headers = reader.next()
        reader.next()
        reader.next()
        reader = sort_qua_by_time(reader)
        time1 = initial_time
        w = []
        x = []
        y = []
        z = []
        for record in reader:
            record_dict = dict(zip(headers, record))
            if getTimeDiff(time1, record_dict["time"]) < timestep:
                w.append(float(record_dict["pepa_qua_w"]))
                x.append(float(record_dict["pepa_qua_x"]))
                y.append(float(record_dict["pepa_qua_y"]))
                z.append(float(record_dict["pepa_qua_z"]))
            else:
                sub_result = []
                sub_result.append(np.mean(w))
                sub_result.append(np.mean(x))
                sub_result.append(np.mean(y))
                sub_result.append(np.mean(z))
                sub_result.append(time1)
                results.append(sub_result)
                del w[:]
                del x[:]
                del y[:]
                del z[:]
                w.append(float(record_dict["pepa_qua_w"]))
                x.append(float(record_dict["pepa_qua_x"]))
                y.append(float(record_dict["pepa_qua_y"]))
                z.append(float(record_dict["pepa_qua_z"]))
                time1 = record_dict["time"]
    CSVWriter(output_file, [['pepa_qua_w', 'pepa_qua_x', 'pepa_qua_y', 'pepa_qua_z', 'time'], ['float', 'float', 'float', 'float', 'datetime'], ['', '', '', '', 'T']])
    CSVWriter(output_file, results)
    
def salt_quaternion(input_file, output_file, timestep, initial_time = '2018-10-12 23:00:00'):
    results = []
    with open(input_file) as fin:
        reader = csv.reader(fin)
        headers = reader.next()
        reader.next()
        reader.next()
        reader = sort_qua_by_time(reader)
        time1 = initial_time
        w = []
        x = []
        y = []
        z = []
        for record in reader:
            record_dict = dict(zip(headers, record))
            if getTimeDiff(time1, record_dict["time"]) < timestep:
                w.append(float(record_dict["salt_qua_w"]))
                x.append(float(record_dict["salt_qua_x"]))
                y.append(float(record_dict["salt_qua_y"]))
                z.append(float(record_dict["salt_qua_z"]))
            else:
                sub_result = []
                sub_result.append(np.mean(w))
                sub_result.append(np.mean(x))
                sub_result.append(np.mean(y))
                sub_result.append(np.mean(z))
                sub_result.append(time1)
                results.append(sub_result)
                del w[:]
                del x[:]
                del y[:]
                del z[:]
                w.append(float(record_dict["salt_qua_w"]))
                x.append(float(record_dict["salt_qua_x"]))
                y.append(float(record_dict["salt_qua_y"]))
                z.append(float(record_dict["salt_qua_z"]))
                time1 = record_dict["time"]
    CSVWriter(output_file, [['salt_qua_w', 'salt_qua_x', 'salt_qua_y', 'salt_qua_z', 'time'], ['float', 'float', 'float', 'float', 'datetime'], ['', '', '', '', 'T']])
    CSVWriter(output_file, results)

In [3]:
# pepa_accerlation('/media/tpc2/DATA/ProcessedLanceData/pepa_acc.csv', '/media/tpc2/DATA/ProcessedLanceData/pepa_acc_timestep3.csv', timestep=3)
# salt_accerlation('/media/tpc2/DATA/ProcessedLanceData/salt_acc.csv', '/media/tpc2/DATA/ProcessedLanceData/salt_acc_timestep3.csv', timestep=3)
# pepa_quaternion('/media/tpc2/DATA/ProcessedLanceData/pepa_qua.csv', '/media/tpc2/DATA/ProcessedLanceData/pepa_qua_timestep3.csv', timestep=3)
# salt_quaternion('/media/tpc2/DATA/ProcessedLanceData/salt_qua.csv', '/media/tpc2/DATA/ProcessedLanceData/salt_qua_timestep3.csv', timestep=3)